# Provider Affiliation
Extract the provider affiliations per healthbase supplemented with CMS data for`coh_prov`

**Script**
* [scripts/de/provider_affiliation.ipynb](./scripts/de/provider_affiliation.ipynb)

**Prior Script(s)**
* [scripts/coh/coh_basic_prov.ipynb](./scripts/coh/coh_basic_prov.ipynb)

**Parameters**
* NONE

**Input**
* `coh_npi`

**Output**  
* `de_provider_affiliation`

**Review**
* [scripts/de/provider_affiliation.html](./scripts/de/provider_affiliation.html)

In [ ]:
#Import libraries for this notebook
import pandas as pd  
from drg_connect import Snowflake
import numpy as np
import pickle
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Load connection variables to connect_dict
with open('../../out/conn/connect_dict.pickle', 'rb') as handle:
    connect_dict = pickle.load(handle)

#Create Eegine to connect to snowflake
snow = Snowflake(role=connect_dict['role'],
                 warehouse=connect_dict['warehouse'],
                 database=connect_dict['database'],
                 schema=connect_dict['schema'])

#Finish engine setup
engine = snow.engine
%load_ext sql_magic
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications


# Extract Data

In [ ]:
%%read_sql
--Create raven diagnosis table
DROP TABLE IF EXISTS de_provider_affiliation;
CREATE TRANSIENT TABLE de_provider_affiliation AS 
  SELECT *
    FROM ref_db.analytics.provider_affiliation
   WHERE provider_npi IN (SELECT npi 
                            FROM coh_npi); 

In [ ]:
%%read_sql
SELECT Count(*) AS row_cnt,
       Count(Distinct provider_npi) AS npi_cnt
  FROM de_provider_affiliation